In [1]:
import numpy as np
import keras
import keras.backend as K

/home/marco/.virtualenvs/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class NALU(keras.layers.Layer):
    
    def __init__(self, units, **kwargs):
        self.units = units
        super(NALU, self).__init__(**kwargs)
    
    def build(self, input_shape):
        self.weight_sigmoid = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform', name='weight_sigmoid')
        self.weight_tanh = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform', name='weight_tanh')
        self.weight_gate = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform', name='weight_gate')
        super(NALU, self).build(input_shape)
    
    def call(self, inputs):
        w = K.sigmoid(self.weight_sigmoid) * K.tanh(self.weight_tanh)
        adds = inputs @ w
        muls = K.exp((K.log(K.abs(inputs) + K.epsilon())) @ w)
        gates = K.sigmoid(inputs @ self.weight_gate)
        return gates * adds + (1-gates) * muls
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

In [35]:
X = X_input = keras.layers.Input((2,))
X = NALU(8)(X)
X = NALU(1)(X)
M = keras.Model(X_input, X)
M.compile('nadam', 'mse')
M.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
nalu_14 (NALU)               (None, 8)                 48        
_________________________________________________________________
nalu_15 (NALU)               (None, 1)                 24        
Total params: 72
Trainable params: 72
Non-trainable params: 0
_________________________________________________________________


In [36]:
def toy_data(a, b):
    return (2*a) / b
data_input = np.random.random(size=(10000,2))*100 + 1
data_label = toy_data(*np.swapaxes(data_input,0,1))

In [37]:
M.fit(data_input, data_label, batch_size=16, epochs=1000, callbacks=[
    keras.callbacks.ReduceLROnPlateau('loss', patience=3, verbose=1),
    keras.callbacks.EarlyStopping('loss', patience=10, verbose=1),
])

Epoch 1/1000
10000/10000 [==============================] - 2s 226us/step - loss: 169.1597
Epoch 2/1000
10000/10000 [==============================] - 2s 201us/step - loss: 134.5190
Epoch 3/1000
10000/10000 [==============================] - 2s 202us/step - loss: 136.7152
Epoch 4/1000
10000/10000 [==============================] - 2s 203us/step - loss: 127.2089
Epoch 5/1000
10000/10000 [==============================] - 2s 203us/step - loss: 117.5212
Epoch 6/1000
10000/10000 [==============================] - 2s 199us/step - loss: 102.5434
Epoch 7/1000
10000/10000 [==============================] - 2s 204us/step - loss: 86.7570
Epoch 8/1000
10000/10000 [==============================] - 2s 197us/step - loss: 72.1196
Epoch 9/1000
10000/10000 [==============================] - 2s 200us/step - loss: 59.0948
Epoch 10/1000
10000/10000 [==============================] - 2s 203us/step - loss: 47.9800
Epoch 11/1000
10000/10000 [==============================] - 2s 205us/step - loss: 39.0884
Ep

10000/10000 [==============================] - 2s 207us/step - loss: 0.0072
Epoch 88/1000
10000/10000 [==============================] - 2s 211us/step - loss: 0.0072
Epoch 89/1000
10000/10000 [==============================] - 2s 202us/step - loss: 0.0072

Epoch 00089: ReduceLROnPlateau reducing learning rate to 2.000000165480742e-10.
Epoch 90/1000
10000/10000 [==============================] - 2s 202us/step - loss: 0.0072
Epoch 91/1000
10000/10000 [==============================] - 2s 205us/step - loss: 0.0072
Epoch 92/1000
10000/10000 [==============================] - 2s 202us/step - loss: 0.0072

Epoch 00092: ReduceLROnPlateau reducing learning rate to 2.000000165480742e-11.
Epoch 93/1000
10000/10000 [==============================] - 2s 206us/step - loss: 0.0072
Epoch 94/1000
10000/10000 [==============================] - 2s 203us/step - loss: 0.0072
Epoch 95/1000
10000/10000 [==============================] - 2s 203us/step - loss: 0.0072

Epoch 00095: ReduceLROnPlateau reducing l

In [46]:
test = np.array([[100, 3]])
print(toy_data(*test[0]))
print(M.predict(test))

66.66666666666667
[[66.33641]]
